# Notebook Overview

This notebook processes article data from a Kafka stream and writes the transformed data to a Delta table. The main steps are:

1. **Read from Kafka**: The source table `udp_prd_atomic.pdp.articles_v2` is assumed to be populated from a Kafka stream containing article data in a nested structure.

2. **Flatten and Transform Data**: A SQL query creates a temporary view `articles_flat` that:
   - Extracts and flattens metadata fields (e.g., `id`, `publisher`, `provenance`, dates).
   - Selects German (`'de'`) or fallback values for fields like `title`, `lead`, and `kicker`.
   - Extracts specific identifiers (e.g., `urn`, `srgId`).
   - Aggregates and flattens arrays (e.g., resources, related articles, contributors, keywords) into CSV strings.
   - Collects URLs from nested resource fields.
   - Preserves Kafka metadata (key, topic, partition, offset, timestamp).

3. **Create DataFrame**: The flattened view is loaded into a Spark DataFrame for further processing or inspection.

4. **Write to Delta Table**: The DataFrame is appended to the Delta table `swi_audience_prd.pdp_articles_v2.articles_v2`, enabling efficient storage and downstream analytics.

Each step ensures that complex, nested Kafka data is transformed into a flat, analytics-ready format.

## Read from Kafka

The following steps read article data from the Delta table `udp_prd_atomic.pdp.articles_v2`, which is assumed to be populated from a Kafka stream. This data includes both the article content and Kafka metadata (such as key, topic, partition, offset, and timestamp). The data is then prepared for further transformation and analysis in subsequent steps.

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW articles_flat AS
WITH base AS (
  SELECT
    key,
    topic,
    partition,
    offset,
    timestamp,
    value AS v
  FROM udp_prd_atomic.pdp.articles_v2
)
SELECT
  -- flache Metadaten
  v.id,
  v.publisher,
  v.provenance,
  v.modificationDate,
  v.releaseDate,

  -- TITLE: erst 'de', sonst erster Eintrag
  coalesce(
    CASE
      WHEN size(filter(coalesce(v.title, array()), x -> x.language = 'de')) > 0
        THEN element_at(transform(filter(v.title, x -> x.language = 'de'), x -> x.content), 1)
    END,
    CASE
      WHEN size(coalesce(v.title, array())) > 0
        THEN element_at(transform(v.title, x -> x.content), 1)
    END
  ) AS title_auto,

  -- LEAD: erst 'de', sonst erster Eintrag
  coalesce(
    CASE
      WHEN size(filter(coalesce(v.lead, array()), x -> x.language = 'de')) > 0
        THEN element_at(transform(filter(v.lead, x -> x.language = 'de'), x -> x.content), 1)
    END,
    CASE
      WHEN size(coalesce(v.lead, array())) > 0
        THEN element_at(transform(v.lead, x -> x.content), 1)
    END
  ) AS lead_auto,

  -- KICKER: erst 'de', sonst erster Eintrag
  coalesce(
    CASE
      WHEN size(filter(coalesce(v.kicker, array()), x -> x.language = 'de')) > 0
        THEN element_at(transform(filter(v.kicker, x -> x.language = 'de'), x -> x.content), 1)
    END,
    CASE
      WHEN size(coalesce(v.kicker, array())) > 0
        THEN element_at(transform(v.kicker, x -> x.content), 1)
    END
  ) AS kicker_auto,

  -- IDs gezielt ziehen (kein Map!)
  CASE
    WHEN size(filter(coalesce(v.identifiers, array()), x -> x.type = 'urn'   AND x.value IS NOT NULL)) > 0
      THEN element_at(transform(filter(v.identifiers, x -> x.type = 'urn'   AND x.value IS NOT NULL), x -> x.value), 1)
  END AS id_urn,

  CASE
    WHEN size(filter(coalesce(v.identifiers, array()), x -> x.type = 'srgId' AND x.value IS NOT NULL)) > 0
      THEN element_at(transform(filter(v.identifiers, x -> x.type = 'srgId' AND x.value IS NOT NULL), x -> x.value), 1)
  END AS id_srg,

  -- Erstes verfügbares Bild
  aggregate(
    coalesce(v.resources, array()),
    CAST(NULL AS STRING),
    (acc, r) -> coalesce(acc, r.picture.locator.url),
    acc -> acc
  ) AS picture_url,

  -- relatedArticles -> CSV
  concat_ws(',', coalesce(v.relatedArticles, CAST(array() AS ARRAY<STRING>))) AS related_articles_csv,

  -- content.text.items -> CSV (Leerzeichen-getrennt)
  CASE
    WHEN v.content IS NOT NULL
      THEN concat_ws(' ', coalesce(v.content.text, CAST(array() AS ARRAY<STRING>)))
    ELSE NULL
  END AS content_text_csv,

  -- contributors -> CSV der Namen
  concat_ws(
    ', ',
    transform(
      coalesce(v.contributors, array()),
      c -> coalesce(
        c.name,
        c.agent.person.name,
        c.agent.team.name,
        c.agent.department.name
      )
    )
  ) AS contributors_csv,

  -- alle locator.url aus resources -> CSV
  concat_ws(
    ',',
    filter(
      flatten(
        transform(
          coalesce(v.resources, array()),
          r -> array(
            r.document.locator.url,
            r.picture.locator.url,
            r.link.locator.url
          )
        )
      ),
      x -> x IS NOT NULL
    )
  ) AS resources_locator_urls_csv,

  -- Keywords -> CSV
  concat_ws(',', coalesce(v.keywords, CAST(array() AS ARRAY<STRING>))) AS keywords_csv,

  -- Kafka-Metadaten (key sicher als STRING)
  CAST(key AS STRING) AS key,
  topic,
  partition,
  offset,
  timestamp
FROM base;


## Create Data Frame

The code below runs a Spark SQL query against the temporary view `articles_flat`, loads the result into a Spark DataFrame named `df`, and then displays the DataFrame for inspection. This step materializes the flattened article data so it can be further processed or written to a Delta table.

In [0]:
df = spark.sql("SELECT * FROM articles_flat")  # or: spark.table("articles_flat")
display(df, truncate=False)

## Write to Delta Table

The following code appends the transformed DataFrame `df` to the Delta table `swi_audience_prd.pdp_articles_v2.articles_v2`. It writes in **append** mode, uses the **Delta** format, and enables **schema merging** so that any new columns are automatically added to the target table without overwriting existing data.

In [0]:
df.write.format("delta") \
    .mode("append") \
    .option("mergeSchema", "true") \
    .saveAsTable("swi_audience_prd.pdp_articles_v2.articles_v2")

In [0]:
import pandas as pd

# Define pandas timestamp bounds
min_ts = pd.Timestamp.min
max_ts = pd.Timestamp.max

df_filtered = df.filter(
    (df["releaseDate"] >= min_ts) & (df["releaseDate"] <= max_ts) &
    (df["modificationDate"] >= min_ts) & (df["modificationDate"] <= max_ts)
)
# save to
df_filtered.toPandas().to_csv(
    "/dbfs/FileStore/tmp/export.csv",
    index=False
)

### Manual Step for Data Sharing on GitHub

**Step 3 – Download the File**

After the export, open the following link in your browser:

https://adb-4119964566130471.11.azuredatabricks.net/files/tmp/export.csv

→ The CSV file will be downloaded ✅  
→ Manually move the file into your GitHub repository folder (e.g., `data/`)  
→ Commit & Push